# About Input/Output Extractor

> 문서 작성일: 2025.03.24

GIP에서 제공하는 평가 기능을 사용할때, 가장 먼저 experiment를 생성해야 합니다. 이는 평가 대상을 지정하는 과정으로서, 문서 작성일을 기준으로 Langfuse에 존재하는 데이터 소스만을 지원하고 있습니다.

- [Dataset](https://api.reference.langfuse.com/?q=observation#tag/datasets/GET/api/public/v2/datasets/{datasetName})
- [Trace](https://api.reference.langfuse.com/?q=observation#tag/trace/GET/api/public/traces)
- [Observation](https://api.reference.langfuse.com/?q=observation#tag/observations/GET/api/public/observations)

하지만 단순히 데이터 소스만을 지정하게 되면 trace나 observation 등에 있는 input/output 데이터를 그대로 활용할 수 밖에 없다는 문제가 있습니다. 이를테면 input 데이터가 다음과 같이 구성되어 있을때, 특정 속성 값만 평가 과정에서 활용하고 싶을 수 있습니다.

```json
{
  "text": "안녕하세요 반갑습니다",
  "label": "인사",
  "user": "nikosyk"
}
```

이런 상황에서는 input/output extractor를 활용하면 됩니다. 이는 input/output 데이터에서 특정 속성 값만 추출해서 평가 과정에서 활용할 수 있도록 하는 기능입니다.

## 사전 준비

우선 다음의 python 패키지가 설치되어 있어야 합니다.

- `requests`
- `python-dotenv`

환경 변수 파일(`.env`)를 사용하기 위해서는 `.env` 파일에 다음과 같은 내용을 채워주세요.

```
GIP_CONSOLE_HOST=https://dev-console-api.platform.a15t.com
GIP_CONSOLE_ENTERPRISE_API_KEY=
GIP_WORKSPACE_ID=
GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID=
```

> 만약 `GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID` 값이 없으면 문서 내의 `Langfuse credential 연동` 부분을 참고해서 설정하시면 됩니다.

In [1]:
import os

from dotenv import load_dotenv


load_dotenv() # Optional: 환경 변수 파일(.env)을 불러옵니다


GIP_CONSOLE_HOST = os.getenv('GIP_CONSOLE_HOST', 'https://dev-console-api.platform.a15t.com')
GIP_CONSOLE_ENTERPRISE_API_KEY = os.getenv('GIP_CONSOLE_ENTERPRISE_API_KEY')
GIP_WORKSPACE_ID = os.getenv('GIP_WORKSPACE_ID')

GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID = os.getenv('GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 8


### 인증 방법 관련 안내

GIP Console의 API를 호출하는 데에는 두가지 방법이 있습니다.

1. Enterprise API Key 사용
2. GIP Console JWT Token 사용

이 문서는 enterprise API key를 사용하는 것을 기준으로 evaluation API를 활용하는 방법에 대해서 안내하고 있으며, 필요하신 경우 인증 헤더 정보를 GIP Console JWT Token을 사용하도록 변경하시면 됩니다.

#### 1. Enterprise API Key 사용

In [ ]:
import requests


response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

#### 2. GIP Console JWT Token 사용

In [ ]:
import requests


# GIP Console 로그인을 통해서 JWT Token을 발급받는 과정
response = requests.post(
    url='https://cognito-idp.ap-northeast-2.amazonaws.com',
    headers={
        'Content-Type': 'application/x-amz-json-1.1',
        'X-Amz-Target': 'AWSCognitoIdentityProviderService.InitiateAuth',
    },
    json={
        'AuthParameters': {
            # GIP Console 로그인 시에 사용하는 계정 정보 기재
            'USERNAME': 'keep-secret',
            'PASSWORD': 'keep-secret',
        },
        'AuthFlow': 'USER_PASSWORD_AUTH',
        'ClientId': '4dm722bemutkce2500cck1bbk4', # dev 환경용 client ID -> prod 환경의 GIP Console API 호출시에는 별도 문의 필요
    },
)

response.raise_for_status()

auth_response = response.json()
id_token = auth_response['AuthenticationResult']['IdToken']

# GIP Console 로그인을 통해서 얻은 JWT Token을 사용해서 GIP Console API 호출
response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'Authorization': f'Bearer {id_token}'}
)

response.raise_for_status()

### Langfuse credential 연동

현재 GIP에서는 Langfuse에 존재하는 데이터 소스에 대한 평가만 우선적으로 지원하기 때문에, Langfuse API key 연동이 필요합니다.

Langfuse의 API key는 각 프로젝트 별로 설정되며, 다음의 문서를 통해서 API key 생성 / 조회 방법을 확인하실 수 있습니다.

[Where are my Langfuse API keys?](https://langfuse.com/faq/all/where-are-langfuse-api-keys)

Langfuse API key를 확인한 이후에는 다음의 과정을 거쳐서 GIP에 연동하실 수 있습니다.

다음의 코드를 환경에 맞게 수정한 이후에 확인되는 결과를 보고, `GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID` 환경 변수 값을 설정하시면 됩니다.

In [ ]:
import requests


response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/integrated-services/credentials',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'service_name': 'LANGFUSE',
        'alias': 'gip-eval-intro-demo', # GIP <-> Langfuse API Key 연동시의 별칭 (Langfuse project name 등으로 설정하시면 편합니다)
        'base_url': 'https://api.langfuse.com', # NOTE: 연동하려는 Langfuse API의 base URL을 입력하세요
        'public_key': '***********',
        'private_key': '***********',
    },
)

response.raise_for_status()

integrated_service_credential = response.json()
integrated_service_credential_id = integrated_service_credential['id']

print('Integrated service credential ID: ', integrated_service_credential_id)

## Input/Output Extractor 사용 방법

Input/Output extractor는 experiment의 소스 데이터 유형에 따라서 다음의 속성 값에 대해 동작하게 됩니다.

| `experiment.type`      | `input extractor`   | `output extractor`            |
|------------------------|---------------------|-------------------------------|
| `langfuse_dataset`     | `DatasetItem.input` | `DatasetItem.expected_output` |
| `langfuse_trace`       | `Trace.input`       | `Trace.output`                |
| `langfuse_observation` | `Observation.input` | `Observation.output`          |

현재 지원되는 extractor 종류는 `json`과 `combined` 두가지가 있습니다. 이는 input/output extractor에 동일하게 적용이 가능합니다.

| `extractor.type` | 설명                                                                          |
|------------------|-----------------------------------------------------------------------------|
| `json`           | JSON 데이터에서 특정 속성 값을 추출합니다                                                   |
| `combined`       | 여러 종류의 extractor를 순차적으로 적용하고(현재는 `json`만 지원), 그 결과를 조합해서 평가에 사용될 데이터를 구성합니다 |

### JSON extractor 사용 방법

간단한 상황에 대해서 input extractor로 `json` 타입을 사용하는 방법에 대해 소개하겠습니다.

#### 예시 상황

1. Langfuse의 dataset을 데이터 소스로 하고, dataset item의 input은 다음과 같은 형식으로 구성됨:
    ```json
    {
      "text": "안녕하세요 반갑습니다",
      "label": "인사",
      "user": "nikosyk"
    }
    ```
2. 평가 과정에서는 `text` 속성 값만을 활용하고 싶음

In [3]:
import requests


response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'experiment-with-input-extractor-demo-temp',
        'description': 'evaluate the extracted input data by input extractor', # Optional
        'type': 'langfuse_dataset',
        'config': {
            'integrated_service_credential_id': GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID,
            'dataset_name': 'extractor-demo',
            'include_archived_items': False,
        },
        'extractors': {
            'input': {
                'type': 'json',
                'config': {
                    'json_path': '.text', # NOTE: `text` property 값을 추출 (`jq` expression을 사용)
                },
            },
            'output': None,
        },
    },
)

response.raise_for_status()

experiment_with_input_extractor = response.json()
experiment_with_input_extractor_id = experiment_with_input_extractor['id']

print('Experiment ID with input extractor:', experiment_with_input_extractor_id)

Experiment ID with input extractor: 0195d1aa-1ad3-7b73-99a8-6a1da9766ef9


`json_path` 값으로는 JSON의 어떤 속성을 추출할 것인지 명시하되, 이는 `jq`의 filter expression을 사용하게 됩니다.

`jq`의 filter expression에 대한 자세한 내용은 [`jq` Manual](https://jqlang.org/manual/#basic-filters)을 참고하시면 됩니다.

이어서 evaluator와 experiment run을 생성해서 평가 결과를 확인해보겠습니다.

In [4]:
judge_prompt = '''
Evaluate the translation quality of a given text by analyzing its translation context. Provide a score from 1 to 3 and a concise rationale for your evaluation referencing the provided context.

# Scoring Criteria

- **1**: The translated text is not understandable or misunderstood due to incomplete translation or mistranslation.
- **2**: Translation has some errors but the meaning is conveyed.
- **3**: Understandable and fully conveys the meaning of the original text.

# Steps

1. Read the original text and the provided translation carefully.
2. Evaluate the translation, considering context and how accurately it conveys the original meaning.
3. Assign a score based on the criteria above.
4. Provide a concise rationale for your score, referencing specific elements of the translation and context.

# Output Format

- **Output**: JSON object including score and rationale
```json
{
  "score": [1, 2, or 3],
  "rationale": "[Your concise rationale here]"
}
```

# Examples

**Example 1:**

- **Original Text**: 팥붕 슈붕 드세요
- **Translated Text**: Please enjoy 팥붕 and 슈붕.
```json
{
  "score": 1,
  "rationale": "Missing translation"
}
```

**Example 2:**

- **Original Text**: 갑엘 덕분에 미국 에이전시와 브랜딩 프로젝트도 너무 잘 마쳤습니다 갑엘 없이 못 살아 정말 못살아
- **Translated Text**: Thanks to Gapel, we successfully completed the branding project with a US agency  I can't live without Gapel, really can't live
```json
{
  "score": 2,
  "rationale": "Mistranslation of proper noun"
}
```

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
'''.strip()

response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/evaluators',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Translation Evaluator',
        'metric': {
            'type': 'category',
            'config': [
                {'index': 0, 'name':  '1'},
                {'index': 1, 'name':  '2'},
                {'index': 2, 'name':  '3'},
            ],
        },
        'type': 'llm_judge',
        'evaluation_config': {
            'judge_model_seq_id': 1676,
            'judge_model_public_id': 'anthropic/claude-3-7-sonnet-20250219',
            'judge_model_parameters': {
                'top_p': None,
                'temperature': 0,
                'max_tokens': 100,
            },
            'judge_prompt': judge_prompt,
        },
    },
)

response.raise_for_status()

evaluator = response.json()
evaluator_id = evaluator['id']

print('Evaluator ID:', evaluator_id)

Evaluator ID: 0195d1aa-2701-75d3-ab01-369a7843887d


In [6]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{experiment_with_input_extractor_id}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': evaluator_id,
        'display_name': 'experiment run with input extractor',
        'target_data_converter': None,
    },
)

response.raise_for_status()

experiment_run_with_input_extractor = response.json()
experiment_run_with_input_extractor_id = experiment_run_with_input_extractor['id']

print('Experiment run ID with input extractor:', experiment_run_with_input_extractor_id)

Experiment run ID with input extractor: 0195d1aa-eb91-7893-9379-c38c02232907


input extractor가 어떻게 동작했는지 확인해보겠습니다.

In [7]:
import csv
import io

response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{experiment_with_input_extractor_id}/runs/{experiment_run_with_input_extractor_id}/download',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

csv_content = response.content.decode('utf-8')
csv_reader = csv.DictReader(io.StringIO(csv_content))

for index, row in enumerate(csv_reader):
    if index > 5: # NOTE: 상세 결과의 일부만 확인
        break

    print('Row #', index)
    print('\texperiment input:\n\t\t', row['input_data_from_experiment'])
    print('\texperiment run input:\n\t\t', row['input_data_from_experiment_run'])
    print('\texperiment output:\n\t\t', row['output_data_from_experiment'])
    print('\texperiment run output:\n\t\t', row['output_data_from_experiment_run'])
    print('\tpassed:', row['passed'])
    print('\terror_message:', row['error_message'] or 'x') # NOTE: `passed` 값이 `false`인 경우에 어떤 문제가 있었는지에 대해 나타냄
    print('\tscore:', row['score'])
    print('\trationale:\n\t\t', row['rationale'])

Row # 0
	experiment input:
		 null
	experiment run input:
		 "안녕하세요 반갑습니다"
	experiment output:
		 null
	experiment run output:
		 {"role": "assistant", "text": "Hello nice to meet you", "model": "openai/gpt-4o"}
	passed: true
	error_message: x
	score: 3
	rationale:
		 The translation accurately conveys the greeting in the original Korean text. '안녕하세요' means 'hello' and '반갑습니다' means 'nice to meet you' or 'pleased to meet you'. The translation captures both parts of the greeting correctly and naturally in English.


### Combined Extractor 사용 방법

Combined extractor는 여러 extractor를 조합해서 순차적으로 적용하여 값을 추출하는 동작을 진행합니다.

문서 작성일을 기준으로, 현재는 JSON extractor만을 지원하고 있기 때문에, `combined` extractor를 사용할 경우에는 JSON extractor를 순차적으로 적용하는 방식으로 동작하게 됩니다.

간단한 상황에 대해서 output extractor로 `combined` 타입을 사용하는 방법에 대해 소개하겠습니다.

#### 예시 상황

1. Langfuse의 dataset을 데이터 소스로 하고, dataset item의 expected output은 다음과 같은 형식으로 구성됨:
    ```json
    {
      "text": "Hello nice to meet you",
      "role": "assistant",
      "model": "openai/gpt-4o"
    }
    ```
2. 평가 과정에서는 `text` 속성 값과 `role` 속성 값을 조합해서 활용하고 싶음

In [11]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'experiment-with-output-extractor-demo-temp',
        'description': 'evaluate the extracted output data by output extractor', # Optional
        'type': 'langfuse_dataset',
        'config': {
            'integrated_service_credential_id': GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID,
            'dataset_name': 'extractor-demo',
            'include_archived_items': False,
        },
        'extractors': {
            'input': None,
            'output': {
                'type': 'combined',
                'config': {
                    'extractors': [
                        {
                            'type': 'json',
                            'config': {
                                'json_path': '.role', # NOTE: `role` property 값을 먼저 추출
                            },
                        },
                        {
                            'type': 'json',
                            'config': {
                                'json_path': '.text', # NOTE: `text` property 값을 그다음으로 추출
                            },
                        },
                    ],
                    # 각 extractor를 통해서 추출한 값을 하나의 문자열로 조합하는 방법을 명시
                    # 별도로 명시하지 않는 경우, `", "`가 기본으로 사용됨 (ex. `{role}, {text}`)
                    # -> 이번 예시에서는 `{role}: {text}`가 됨
                    'join_with': ': ',
                }
            },
        },
    },
)

response.raise_for_status()

experiment_with_output_extractor = response.json()
experiment_with_output_extractor_id = experiment_with_output_extractor['id']

print('Experiment ID with output extractor:', experiment_with_output_extractor_id)

Experiment ID with output extractor: 0195d1b1-b7ed-7d11-afcc-9bc87a418c8b


In [15]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{experiment_with_output_extractor_id}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': evaluator_id,
        'display_name': 'experiment run with output extractor',
        'target_data_converter': None,
    },
)

response.raise_for_status()

experiment_run_with_output_extractor = response.json()
experiment_run_with_output_extractor_id = experiment_run_with_output_extractor['id']

print('Experiment run ID with output extractor:', experiment_run_with_output_extractor_id)

Experiment run ID with output extractor: 0195d1b3-0779-7a31-886b-7e87f49b427a


output extractor가 어떻게 동작했는지 확인해보겠습니다.

In [17]:
response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{experiment_with_output_extractor_id}/runs/{experiment_run_with_output_extractor_id}/download',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

csv_content = response.content.decode('utf-8')
csv_reader = csv.DictReader(io.StringIO(csv_content))

for index, row in enumerate(csv_reader):
    if index > 5: # NOTE: 상세 결과의 일부만 확인
        break

    print('Row #', index)
    print('\texperiment input:\n\t\t', row['input_data_from_experiment'])
    print('\texperiment run input:\n\t\t', row['input_data_from_experiment_run'])
    print('\texperiment output:\n\t\t', row['output_data_from_experiment'])
    print('\texperiment run output:\n\t\t', row['output_data_from_experiment_run'])
    print('\tpassed:', row['passed'])
    print('\terror_message:', row['error_message'] or 'x') # NOTE: `passed` 값이 `false`인 경우에 어떤 문제가 있었는지에 대해 나타냄
    print('\tscore:', row['score'])
    print('\trationale:\n\t\t', row['rationale'])

Row # 0
	experiment input:
		 null
	experiment run input:
		 {"text": "안녕하세요 반갑습니다", "user": "nikosyk", "label": "인사"}
	experiment output:
		 null
	experiment run output:
		 "assistant: Hello nice to meet you"
	passed: true
	error_message: x
	score: 3
	rationale:
		 The translation accurately conveys the greeting 'Hello nice to meet you' from the Korean '안녕하세요 반갑습니다', which is a standard greeting meaning 'Hello, nice to meet you.' The meaning is fully preserved and naturally expressed in English.


의도했던 것처럼 `{role}: {text}` 형태로 값이 잘 추출되어 `assistant: Hello nice to meet you`로 output 값이 출력되는 것을 확인할 수 있습니다.